![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# What is Context Engineering?

## Introduction

**Context Engineering** is the discipline of designing, implementing, and optimizing context management systems for AI agents and applications. It's the practice of ensuring that AI systems have the right information, at the right time, in the right format to make intelligent decisions and provide relevant responses.

Think of context engineering as the "memory and awareness system" for AI agents - it's what allows them to:
- Remember past conversations and experiences
- Understand their role and capabilities
- Access relevant information from large knowledge bases
- Maintain coherent, personalized interactions over time

## Why Context Engineering Matters

Without proper context engineering, AI agents are like people with severe amnesia - they can't remember what happened five minutes ago, don't know who they're talking to, and can't learn from experience. This leads to:

❌ **Poor User Experience**
- Repetitive conversations
- Lack of personalization
- Inconsistent responses

❌ **Inefficient Operations**
- Redundant processing
- Inability to build on previous work
- Lost context between sessions

❌ **Limited Capabilities**
- Can't handle complex, multi-step tasks
- No learning or adaptation
- Poor integration with existing systems

## Core Components of Context Engineering

Context engineering involves several key components working together:

### 1. **System Context**
What the AI should know about itself and its environment:
- Role and responsibilities
- Available tools and capabilities
- Operating constraints and guidelines
- Domain-specific knowledge

### 2. **Memory Management**
How information is stored, retrieved, and maintained:
- **Working memory**: Persistent storage focused on the current task, including conversation context and task-related data
- **Long-term memory**: Knowledge learned across sessions, such as user preferences and important facts

### 3. **Context Retrieval**
How relevant information is found and surfaced:
- Semantic search and similarity matching
- Relevance ranking and filtering
- Context window management

### 4. **Context Integration**
How different types of context are combined:
- Merging multiple information sources
- Resolving conflicts and inconsistencies
- Prioritizing information by importance

## Real-World Example: University Class Agent

Let's explore context engineering through a practical example - a university class recommendation agent. This agent helps students find courses, plan their academic journey, and provides personalized recommendations.

### Without Context Engineering
```
Student: "I'm interested in programming courses"
Agent: "Here are all programming courses: CS101, CS201, CS301..."

Student: "I prefer online courses"
Agent: "Here are all programming courses: CS101, CS201, CS301..."

Student: "What about my major requirements?"
Agent: "I don't know your major. Here are all programming courses..."
```

### With Context Engineering
```
Student: "I'm interested in programming courses"
Agent: "Great! I can help you find programming courses. Let me search our catalog...
        Based on your Computer Science major and beginner level, I recommend:
        - CS101: Intro to Programming (online, matches your preference)
        - CS102: Data Structures (hybrid option available)"

Student: "Tell me more about CS101"
Agent: "CS101 is perfect for you! It's:
        - Online format (your preference)
        - Beginner-friendly
        - Required for your CS major
        - No prerequisites needed
        - Taught by Prof. Smith (highly rated)"
```

## Environment Setup

Before we explore context engineering in action, let's set up our environment with the necessary dependencies and connections.

In [3]:
# Install the Redis Context Course package
%pip install --upgrade -q -e ../../reference-agent


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Required API Keys

In [2]:
import os
import getpass

# This example needs an OpenAI key to run
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

In [ ]:
# Setup Redis (uncomment if running in Colab)
# !curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# !echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# !sudo apt-get update > /dev/null 2>&1
# !sudo apt-get install redis-server > /dev/null 2>&1
# !redis-server --daemonize yes

# Set Redis URL
os.environ["REDIS_URL"] = "redis://localhost:6379"

In [4]:
# Import the Redis Context Course components
from redis_context_course.models import Course, StudentProfile, DifficultyLevel, CourseFormat
from redis_context_course import MemoryClient
from redis_context_course.course_manager import CourseManager
from redis_context_course.redis_config import redis_config

# Check Redis connection
redis_available = redis_config.health_check()
print(f"Redis connection: {'✅ Connected' if redis_available else '❌ Failed'}")
print("✅ Redis Context Course package imported successfully")

Redis connection: ✅ Connected
✅ Redis Context Course package imported successfully


## Context Engineering in Action

Now that our environment is ready, let's explore the different types of context our agent manages:

### 1. System Context Example

System context defines what the agent knows about itself. This is typically provided as a system prompt:

In [5]:
# Example of a system prompt - the agent's instructions and constraints
system_prompt = """
You are a helpful university class recommendation agent for Redis University.
Your role is to help students find courses, plan their academic journey, and
answer questions about the course catalog.

## Your Responsibilities

- Help students discover courses that match their interests and goals
- Provide accurate information about course content, prerequisites, and
  schedules
- Remember student preferences and use them to personalize recommendations
- Guide students toward courses that align with their major requirements

## Important Constraints

- Only recommend courses that exist in the course catalog (use the
  search_courses tool to verify)
- Always check prerequisites before recommending a course
- Respect student preferences for course format (online, in-person, hybrid)
- Be honest when you don't know something - don't make up course information
- If a student asks about a course that doesn't exist, help them find similar
  alternatives

## Interaction Guidelines

- Be friendly, encouraging, and supportive
- Ask clarifying questions when student requests are vague
- Explain your reasoning when making recommendations
- Keep responses concise but informative
- Use the student's name when you know it

## Tools Available

You have access to tools for searching the course catalog and managing student
memories. Use these tools to provide accurate, personalized recommendations.
"""

print("🤖 System Prompt Example:")
print("=" * 60)
print(system_prompt)
print("=" * 60)
print("\nThis system prompt will be included in every conversation turn,")
print("giving the LLM consistent instructions about its role and behavior.")

🤖 System Prompt Example:

You are a helpful university class recommendation agent for Redis University.
Your role is to help students find courses, plan their academic journey, and
answer questions about the course catalog.

## Your Responsibilities

- Help students discover courses that match their interests and goals
- Provide accurate information about course content, prerequisites, and
  schedules
- Remember student preferences and use them to personalize recommendations
- Guide students toward courses that align with their major requirements

## Important Constraints

- Only recommend courses that exist in the course catalog (use the
  search_courses tool to verify)
- Always check prerequisites before recommending a course
- Respect student preferences for course format (online, in-person, hybrid)
- Be honest when you don't know something - don't make up course information
- If a student asks about a course that doesn't exist, help them find similar
  alternatives

## Interaction 

### 2. Student Context Example

Student context represents what the agent knows about the user:

In [6]:
# Example student profile - user context
student = StudentProfile(
    name="Alex Johnson",
    email="alex.johnson@university.edu",
    major="Computer Science",
    year=2,
    completed_courses=["CS101", "MATH101", "ENG101"],
    current_courses=["CS201", "MATH201"],
    interests=["machine learning", "web development", "data science"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=15
)

print("👤 Student Context:")
print(f"Name: {student.name}")
print(f"Major: {student.major} (Year {student.year})")
print(f"Completed: {len(student.completed_courses)} courses")
print(f"Current: {len(student.current_courses)} courses")
print(f"Interests: {', '.join(student.interests)}")
print(f"Preferences: {student.preferred_format.value}, {student.preferred_difficulty.value} level")
import os

👤 Student Context:
Name: Alex Johnson
Major: Computer Science (Year 2)
Completed: 3 courses
Current: 2 courses
Interests: machine learning, web development, data science
Preferences: online, intermediate level


### 3. Memory Context Example

Memory context includes past conversations and stored knowledge. Our agent uses the Agent Memory Server to store and retrieve memories.

**Note:** This requires the Agent Memory Server to be running. See Section 3 notebooks for detailed memory operations.

In [ ]:

from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig
from agent_memory_client.models import MemoryTypeEnum
from agent_memory_client.models import ClientMemoryRecord

# Initialize memory client
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)

# Example of storing different types of memories
async def demonstrate_memory_context():
    await memory_client.create_long_term_memory([
        ClientMemoryRecord(
            text="I prefer online courses because I work part-time",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["preferences", "schedule"]
        ),
        ClientMemoryRecord(
            text="I want to specialize in machine learning and AI",
            memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["goals", "career"]
        ),
       ClientMemoryRecord(
            text="Student struggled with calculus but excelled in programming courses",
           memory_type=MemoryTypeEnum.SEMANTIC,
            topics=["academic_performance", "strengths"]
    )])
    
    print("🧠 Memory Context Stored:")
    print("✅ Preference stored")
    print("✅ Goal stored")
    print("✅ Academic performance noted")
    
    # Retrieve relevant memories using semantic search
    results = await memory_client.search_long_term_memory(
        text="course recommendations for machine learning",
        namespace={"eq": "redis_university"},
        limit=3
    )
    
    print(f"\n🔍 Retrieved {len(results.memories)} relevant memories:")
    for memory in results.memories:
        print(f"  • [{memory.memory_type}] {memory.text[:60]}...")

# Run the memory demonstration
await demonstrate_memory_context()

## Context Integration in Practice

Now let's see how all these context types work together to construct the actual prompt sent to the LLM:

In [ ]:
async def demonstrate_context_integration():
    """
    This demonstrates how we assemble different context sources into a complete prompt.
    """
    print("🎯 Context Integration: Building the Complete Prompt")
    print("=" * 70)

    # 1. Student asks for recommendations
    user_query = "What courses should I take next semester?"
    print(f"\n📝 User Query: '{user_query}'")

    # 2. Retrieve relevant memories
    print("\n🔍 Step 1: Searching long-term memory...")
    memory_results = await memory_client.search_long_term_memory(
        text=user_query,
        namespace={"eq": "redis_university"},
        limit=3
    )

    # Format memories for the prompt
    memories_text = "\n".join([
        f"- {memory.text}"
        for memory in memory_results.memories
    ]) if memory_results.memories else "No relevant memories found."

    print(f"   Found {len(memory_results.memories)} relevant memories")

    # 3. Get student profile information
    print("\n👤 Step 2: Loading student profile...")
    # Using the student profile we created earlier
    student_context = f"""Name: {student.name}
Major: {student.major} (Year {student.year})
Completed Courses: {', '.join(student.completed_courses)}
Current Courses: {', '.join(student.current_courses)}
Interests: {', '.join(student.interests)}
Preferred Format: {student.preferred_format.value}
Preferred Difficulty: {student.preferred_difficulty.value}"""

    print("   Profile loaded")

    # 4. Assemble the complete prompt
    print("\n🔧 Step 3: Assembling complete prompt...")

    # This is the actual prompt that would be sent to the LLM
    complete_prompt = f"""SYSTEM PROMPT:
{system_prompt}

STUDENT PROFILE:
{student_context}

POTENTIALLY RELEVANT MEMORIES:
{memories_text}

USER QUERY:
{user_query}

Please provide a helpful response based on the student's profile, memories, and query."""

    # 5. Display the assembled prompt
    print("\n" + "=" * 70)
    print("📋 COMPLETE ASSEMBLED PROMPT (sent to LLM):")
    print("=" * 70)
    print(complete_prompt)
    print("=" * 70)

    print("\n💡 Key Points:")
    print("   • System prompt defines the agent's role and constraints")
    print("   • Student profile provides current context about the user")
    print("   • Memories add relevant information from past conversations")
    print("   • User query is the current request")
    print("   • All assembled into a single prompt for the LLM")

await demonstrate_context_integration()


## Key Takeaways

From this introduction to context engineering, we can see several important principles:

### 1. **Context is Multi-Dimensional**
- **System context**: What the AI knows about itself
- **User context**: What the AI knows about the user
- **Domain context**: What the AI knows about the subject matter
- **Conversation context**: What has been discussed recently
- **Historical context**: What has been learned over time

Some of these sources are static, updated only when the agent's code changes,
while others may be retrieved dynamically from external sources, such as
via APIs or vector search.

### 2. **Memory is Essential**
- **Working memory**: Maintains conversation flow and task-related context
- **Long-term memory**: Enables learning and personalization across sessions

### 3. **Context Must Be Actionable**
- Information is only valuable if it can improve responses
- Context should be prioritized by relevance and importance -- this is often done through scoring and filtering
- The system must be able to integrate multiple context sources

### 4. **Context Engineering is Iterative**
- Systems improve as they gather more context -- though as we'll see in the course, you there are limits
- Context quality affects response quality
- Feedback loops help refine context management

## Next Steps

In the next notebook, we'll explore **The Role of a Context Engine** - the technical infrastructure that makes context engineering possible. We'll dive deeper into:

- Vector databases and semantic search
- Memory architectures and storage patterns
- Context retrieval and ranking algorithms
- Integration with LLMs and agent frameworks

## Try It Yourself

Experiment with the concepts we've covered:

1. **Modify the student profile** - Change interests, preferences, or academic history
2. **Add new memory types** - Store different kinds of information
3. **Experiment with context retrieval** - Try different queries and see what memories are retrieved
4. **Think about your own use case** - How would context engineering apply to your domain?

The power of context engineering lies in its ability to make AI systems more intelligent, personalized, and useful. As we'll see in the following notebooks, the technical implementation of these concepts using Redis, LangGraph, and modern AI tools makes it possible to build sophisticated, context-aware applications.